In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# Overview

Vamos fazer uma análise das vendas por loja e departamentos do Walmart. 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib
import time
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('seaborn-darkgrid')
import statsmodels.api as sm
matplotlib.rcParams['axes.labelsize'] = 20
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [ ]:
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn import preprocessing
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor 
from sklearn.model_selection import GridSearchCV

Vamos separar os data sets e analisar as informações que estão contidas em cada um

In [ ]:
filepath = '../input/walmart-recruiting-store-sales-forecasting/'

In [ ]:
dados = pd.read_csv(f'{filepath}train.csv.zip', parse_dates=['Date'], compression='zip')
dados.info()

In [ ]:
lojas = pd.read_csv(f'{filepath}stores.csv')
lojas.info()

In [ ]:
features = pd.read_csv(f'{filepath}features.csv.zip', parse_dates=['Date'], compression='zip')
features.info()

In [ ]:
predizer = pd.read_csv(f'{filepath}test.csv.zip', parse_dates=['Date'], compression='zip')
predizer.info()

# Análise de Variáveis

In [ ]:
plt.figure(figsize=(10,8))
sns.distplot(dados.Weekly_Sales)

In [ ]:
plt.figure(figsize=(10,8))
sns.boxplot(x="Type", y="Size", data=lojas)

As classificações de tipos de lojas parece ser feito de acordo com o tamanho das lojas.
Vamos agrupar as informações de features e das lojas para continuar as análises de variáveis.

In [ ]:
valores_por_loja_tamanho = dados.merge(lojas, on='Store', how='left')
lojas_completas = valores_por_loja_tamanho.merge(features, on=['Store','Date'], how='left')
lojas_completas.info()

In [ ]:
plt.figure(figsize=(20,6))
sns.boxplot(x="Store", y="Weekly_Sales",hue='IsHoliday_x', data=lojas_completas)

Como nossos dados estão granulares no nível de departamento, vamos agrupar por loja para tentar identificar melhor as tendências

In [ ]:
vendas_no_tempo = lojas_completas.groupby(['Date','Type']).mean()['Weekly_Sales'].reset_index()

In [ ]:
plt.figure(figsize=(30,8))
sns.lineplot(x="Date", y="Weekly_Sales",hue='Type', data=vendas_no_tempo)

Podemos verificar que as lojas de tipo C tem uma variabilidade menor durante o tempo, talvez essas lojas não tenham tanta influência dos feriados, vamos analisar mais a fundo.

In [ ]:
vendas_agrupadas = lojas_completas.groupby(['Store','Date','Type','IsHoliday_x']).sum()['Weekly_Sales'].reset_index()

In [ ]:
plt.figure(figsize=(20,6))
sns.boxplot(x="Store", y="Weekly_Sales",hue='IsHoliday_x', data=vendas_agrupadas)

Podemos identificar que existe alguma flutuação nas vendas em feriados, vamos colocar os feriados fornecidos para identificar possíveis interações.

In [ ]:
feriados = {
    'Date' : (pd.to_datetime(['2010-02-12', '2011-02-11', '2012-02-10', '2013-02-08',
                             '2010-09-10', '2011-09-09', '2012-09-07', '2013-09-06',
                            '2010-11-26', '2011-11-25', '2012-11-23', '2013-11-29',
                            '2010-12-31', '2011-12-30', '2012-12-28', '2013-12-27'])),
    'TypeHoliday' :(['SB', 'SB', 'SB', 'SB',
                    'Labor', 'Labor', 'Labor', 'Labor',
                    'Thanksgiving', 'Thanksgiving', 'Thanksgiving', 'Thanksgiving',
                    'Christmas', 'Christmas', 'Christmas', 'Christmas']) 

}

In [ ]:
lojas_feriados = lojas_completas.merge(pd.DataFrame(feriados), on='Date', how='left')
lojas_feriados['TypeHoliday'].fillna(0,inplace=True)
lojas_feriados

Vamos extrair as colunas de mês e ano para identificar se os feriados tem comportamento diferente no tempo

In [ ]:
lojas_feriados['Month'] = pd.to_datetime(lojas_feriados.Date).dt.month
lojas_feriados['Year'] = pd.to_datetime(lojas_feriados.Date).dt.year

In [ ]:
plot_feriados = lojas_feriados.groupby(['Store','Year','TypeHoliday','Type']).sum()['Weekly_Sales'].reset_index()
plt.figure(figsize=(15,8))
sns.boxplot(x="Year", y="Weekly_Sales",hue='TypeHoliday', data=plot_feriados.loc[plot_feriados['TypeHoliday']!=0])

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot(x="Type", y="Weekly_Sales",hue='TypeHoliday', data=plot_feriados.loc[plot_feriados['TypeHoliday']!=0])

Podemos ver que os feriados de Ação de Graças tem um valor de vendas superior aos demais feriados. Vamos considerar isso nas predições.
Também certificamos que as lojas do tipo C não tem forte influência dos feriados.

### Tratando MissingValues

Antes de começar os testes de modelos, vamos tratar os valores faltantes. 
Nota-se que os MarkDowns são os unicos valores faltantes neste dataset, e considerando a documentação e a natureza dessas features vamos fazer uma rápida análise para verificar o comportamento dessas variáveis.

In [ ]:
plt.figure(figsize=(30,8))
sns.boxplot(x="Store", y="MarkDown1",hue='IsHoliday_x', data=lojas_feriados)

In [ ]:
plt.figure(figsize=(30,8))
sns.boxplot(x="Store", y="MarkDown2",hue='IsHoliday_x', data=lojas_feriados)

In [ ]:
plt.figure(figsize=(30,8))
sns.boxplot(x="Store", y="MarkDown3",hue='IsHoliday_x', data=lojas_feriados)

In [ ]:
plt.figure(figsize=(30,8))
sns.boxplot(x="Store", y="MarkDown4",hue='IsHoliday_x', data=lojas_feriados)

In [ ]:
plt.figure(figsize=(30,8))
sns.boxplot(x="Store", y="MarkDown5",hue='IsHoliday_x', data=lojas_feriados)

Notamos que os MarkDowns tem bastantes outliers por loja, por isso iremos tratar os missing values utilizando as medianas dos valores por loja e para caso tenha feriado ou não na data.

In [ ]:
filling = lojas_feriados.groupby(['Store','IsHoliday_x']).median()[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].reset_index()
filling.rename(columns={'MarkDown1':'FMD1','MarkDown2':'FMD2','MarkDown3':'FMD3','MarkDown4':'FMD4','MarkDown5':'FMD5'}, inplace=True)

In [ ]:
lojas_preenchidas = lojas_feriados.merge(filling, on=['Store','IsHoliday_x'], how='inner')

In [ ]:
lojas_preenchidas.MarkDown1.fillna(lojas_preenchidas['FMD1'],inplace=True)
lojas_preenchidas.MarkDown2.fillna(lojas_preenchidas['FMD2'],inplace=True)
lojas_preenchidas.MarkDown3.fillna(lojas_preenchidas['FMD3'],inplace=True)
lojas_preenchidas.MarkDown4.fillna(lojas_preenchidas['FMD4'],inplace=True)
lojas_preenchidas.MarkDown5.fillna(lojas_preenchidas['FMD5'],inplace=True)
lojas_preenchidas.drop(['FMD1','FMD2','FMD3','FMD4','FMD5'], axis=1, inplace=True)
lojas_preenchidas.info()

Como verificamos que os feriados tem diferentes pesos nas vendas das lojas, vamos separalos em variáveis para auxiliar as nossas análises.

In [ ]:
lojas_preenchidas['IsHoliday'] = pd.get_dummies(lojas_preenchidas.IsHoliday_x)[1]
lojas_preenchidas['SB'] =pd.get_dummies(lojas_preenchidas.TypeHoliday)['SB']
lojas_preenchidas['Labor'] =pd.get_dummies(lojas_preenchidas.TypeHoliday)['Labor']
lojas_preenchidas['Thanksgiving'] =pd.get_dummies(lojas_preenchidas.TypeHoliday)['Thanksgiving']
lojas_preenchidas['Christmas'] =pd.get_dummies(lojas_preenchidas.TypeHoliday)['Christmas']
lojas_preenchidas['TypeA'] = pd.get_dummies(lojas_preenchidas.Type)['A']
lojas_preenchidas['TypeB'] = pd.get_dummies(lojas_preenchidas.Type)['B']
lojas_preenchidas['TypeC'] = pd.get_dummies(lojas_preenchidas.Type)['C']
lojas_preenchidas.head()

Com todas as variáveis preenchidas no DataSet, vamos analisar as possíveis correlações entre elas.

In [ ]:
plt.figure(figsize=(25,20))
sns.heatmap(lojas_preenchidas.fillna(0).corr(), annot=True)

Podemos verificar algumas correlações lógicas entre feriados, mas também podemos notar algumas correlações entre MarkDowns. Vamos retirar algumas variáveis afim de simplificar o modelo e mitigar problemas de multicolinearidade dado que ainda não decidimos qual modelo vamos utilizar em nossas predições.

Uma ultima análise, vimos que as vendas das lojas C não se comportam da mesma maneira que as demais lojas, portanto vamos verificar como elas se comportam, dividindo a loja pelo seu tamanho. 

In [ ]:
plt.figure(figsize=(20,10))
lojas_preenchidas['Weekly_Sales_Size'] = lojas_preenchidas['Weekly_Sales']/lojas_preenchidas['Size']
sns.lineplot(x="Date", y="Weekly_Sales_Size",hue='Type', data=lojas_preenchidas)

As lojas C tem um comportamento bem diferente, o que pode causar alguns problemas em nosso modelo. Vamos manter as variáveis de tipo afim de tentar reduzir esse problema.

### Seleção de Variáveis.

Vamos retirar as variáveis com alguma colinearidade com os MarkDonw, além das variáveis categóricas e da Weekly_Sale_Size que serviu apenas para uma análise pontual do comportamento.

In [ ]:
lojas_limpas = lojas_preenchidas.drop(['MarkDown4','MarkDown5','IsHoliday_y','IsHoliday_x','Type','TypeHoliday','TypeC','Weekly_Sales_Size'], axis=1)

# Modelo

A primeira vista, considerando as vendas durante um determinado período de tempo para prever vendas futuras, foi considerado a utilização da abordagem por <b> Séries Temporais </b>. Para tal deveríamos analisar as vendas no tempo e tentar ajusar um modelo de classe ARIMAX para projetar vendas futuras. 
Analisando as features, notamos que existem fetures de datas futuras, ou datas as quais iremos estimar, dados estas informações podemos utiliza algum modelo de <b> Regressão/Classificação </b>.
Caso fossemos utilizar um modelo ARIMAX, tirariamos as variáveis de departamento, buscando analisar apenas as vendas semanais. 
Fizemos uma pequena análise das vendas e percebemos que as diferenças entre semanas tem um comportamento estacionário, o que permitiria uma abordagem de séries temporais, o que não foi o caso selecionado.

## Treino e Teste

Mesmo não utilizando a abordagem de séries temporais, vamos fazer uma quebra de treino e teste considerando que nossas vendas se comportam com uma função do tempo, para isso vamos quebrar a série de Teste em <b> Out of Sample </b> e <b> Out of Time </b>

Como estamos falando em vendas semanais, vamos separar um mês de vendas para nossa análise out of time.

In [ ]:
OfT = lojas_limpas.loc[lojas_limpas['Date'] >= '2012-10-05']
lojas_predict = lojas_limpas.loc[lojas_limpas['Date'] < '2012-10-05']

Agora vamos quebrar nossas variáveis nos X e Y que utilizaremos para treinar os modelos.

In [ ]:
# Conjunto OfT
X_OfT = OfT.drop(['Weekly_Sales','Date'], axis=1)
y_OfT = OfT['Weekly_Sales']
# Conjunto OfS
X = lojas_predict.drop(['Weekly_Sales','Date'], axis=1)
y = lojas_predict['Weekly_Sales']

In [ ]:
X_OfT.head()

Para coparar os modelos que iremos ajustar as predições usando a função determinada pelo desafio de WMAE

In [ ]:
def WMAE(y_obs, y_pred, flag):
    peso = flag*5+1*(1-flag)
    
    indice = (1/sum(peso))*sum(peso*abs(y_obs-y_pred))

    return indice

Quebrando os conjuntos de treino e teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
print(f'Formato do conjunto de treino OfS: {X_train.shape}')
print(f'Formato do conjunto de teste: {X_test.shape}')

Mesma quantidade de variáveis, ok. 

## Os Modelos

Foram escolhidos três modelos para comparar performance e verificar qual seria o modelo adotado para esse caso: LinearRegression, RandomForestRegressor e GradientBoostRegressor. Cada um dos tres modelos tem uma especificidade para como fitam os dados, portanto vamos olhar todos.
A <b> Regressão Linear </b> nos servirá como baseline para a performance dos modelos.

Vamos usar o método de CrossValidation para veirificar a coerência dos nossos modelos, assim como sua estabilidade nos conjuntos de dados. 
Usaremos também o GridSearch para vefiricar quais os parâmentros ótimos de cada modelo que estamos utilizando.

### Regressão Linear

In [ ]:
parameters = {'normalize': [False,True], 'fit_intercept': [False,True]} 
model = LinearRegression()

SEED = 1988
np.random.seed(SEED)
cv = KFold(10, shuffle=True)



clf = GridSearchCV(model, parameters, cv=cv, verbose=5, n_jobs=8)
clf.fit(X_train, y_train)
clf.best_params_
clf.predict(X_test)
wmae = WMAE(y_test, clf.predict(X_test), X_test.IsHoliday)
r2 = r2_score(y_test, clf.predict(X_test))
results = cross_validate(model, X, y, cv=cv, return_train_score = False)
media = results['test_score'].mean()
desvio = results['test_score'].std()

print("LinearRegression")
print("------------------------------")
print(f'Parametros ótimos = {clf.best_params_}')
print(f'Mean: {media*100}')
print(f'Accuracy: [{(media-2*desvio)*100} , {(media+2*desvio)*100}]')
print(f'WMAE = {wmae} and R-square = {r2}')
print("------------------------------")

Veremos que o nosso baseline é em 14.910, vamos verificar como os outros modelos se comportam. 

* ### Random Forest Regressor e Gradient Boosting Regressor

Foi feita uma rodada de GridSearch, para determinar os melhores parâmentros dos modelos em questão. Com esses parâmetros calibraremos e faremos as predições e análise dos dados. 

### Calibrando e Calculando a Efetividade dos Modelos

In [ ]:
SEED = 1988
np.random.seed(SEED)
ln = LinearRegression(fit_intercept = True, normalize = False)
rf = RandomForestRegressor(max_depth = 30, max_features =  18, min_samples_leaf = 2 ,n_estimators = 200)
gbm = GradientBoostingRegressor(max_depth = 5, max_features = 12, min_samples_leaf = 5, n_estimators = 200)
np.random.seed(SEED)

ln.fit(X_train, y_train)
ln_y_pred = ln.predict(X_test)
wmae_ln_pred = WMAE(y_test, ln_y_pred, X_test.IsHoliday)
r2_ln_pred = r2_score(y_test, ln_y_pred)

rf.fit(X_train, y_train)
rf_y_pred = rf.predict(X_test)
wmae_rf_pred = WMAE(y_test, rf_y_pred, X_test.IsHoliday)
r2_rf_pred = r2_score(y_test, rf_y_pred)

gbm.fit(X_train, y_train)
gbm_y_pred = gbm.predict(X_test)
wmae_gbm_pred = WMAE(y_test, gbm_y_pred, X_test.IsHoliday)
r2_gbm_pred = r2_score(y_test, gbm_y_pred)

print("LinearRegression")
print("------------------------------")
print(f'WMAE = {wmae_ln_pred} and R-square = {r2_ln_pred}')
print("------------------------------")
print("RandomForest")
print("------------------------------")
print(f'WMAE = {wmae_rf_pred} and R-square = {r2_rf_pred}')
print("------------------------------")
print("GradientBoosting")
print("------------------------------")
print(f'WMAE = {wmae_gbm_pred} and R-square = {r2_gbm_pred}')
print("------------------------------")

In [ ]:
plt.figure(figsize = (20,12))
plt.scatter(y_test,ln_y_pred,label='LR',marker = 'o',color='r')
plt.scatter(y_test,rf_y_pred,label='RF',marker = 'o',color='b')
plt.scatter(y_test,gbm_y_pred,label='GBR',marker = 'o',color='y')
plt.title('Modelos',fontsize = 25)
plt.legend(fontsize = 20)
plt.show()

O Modelo <b> Random Forest Regressor </b>, parece mais consistente nesse cenário, vamos analisar a amostra <b>OfT</b>.

### Teste Out of Time

In [ ]:
ln_y_OfT = ln.predict(X_OfT)
wmae_ln_OfT = WMAE(y_OfT, ln_y_OfT, X_OfT.IsHoliday)
r2_ln_OfT = r2_score(y_OfT, ln_y_OfT)


rf_y_OfT = rf.predict(X_OfT)
wmae_rf_OfT = WMAE(y_OfT, rf_y_OfT, X_OfT.IsHoliday)
r2_rf_OfT = r2_score(y_OfT, rf_y_OfT)


gbm_y_OfT = gbm.predict(X_OfT)
wmae_gbm_OfT = WMAE(y_OfT, gbm_y_OfT, X_OfT.IsHoliday)
r2_gbm_OfT = r2_score(y_OfT, gbm_y_OfT)

print("LinearRegression")
print("------------------------------")
print(f'WMAE = {wmae_ln_OfT} and R-square = {r2_ln_OfT}')
print("------------------------------")
print("RandomForest")
print("------------------------------")
print(f'WMAE = {wmae_rf_OfT} and R-square = {r2_rf_OfT}')
print("------------------------------")
print("GradientBoosting")
print("------------------------------")
print(f'WMAE = {wmae_gbm_OfT} and R-square = {r2_gbm_OfT}')
print("------------------------------")

In [ ]:
plt.figure(figsize = (20,12))
plt.scatter(y_OfT,ln_y_OfT,label='LR',marker = 'o',color='r')
plt.scatter(y_OfT,rf_y_OfT,label='RF',marker = 'o',color='b')
plt.scatter(y_OfT,gbm_y_OfT,label='GBR',marker = 'o',color='y')
plt.title('Modelos',fontsize = 25)
plt.legend(fontsize = 20)
plt.show()

O modelo <b> Random Forest </b> se mostrou mais consistente nos testes com estes parâmetros. O <b> Gradient Boosting Regressor </b> teve uma efetividade melhor em testes com outros parâmetros, que demandam muito mais do nosso poder computacional, portanto optamos por não utiliza-lo.

# Predições

In [ ]:
predicoes_loja = predizer.merge(features, on=['Store','Date'], how='inner')

In [ ]:
lojas_p_features = predicoes_loja.merge(lojas, on='Store', how='inner')

In [ ]:
filling_p = lojas_p_features.groupby(['Store','IsHoliday_x','Type']).median()[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','CPI','Unemployment']].reset_index()
filling_p.rename(columns={'MarkDown1':'FMD1','MarkDown2':'FMD2','MarkDown3':'FMD3','MarkDown4':'FMD4','MarkDown5':'FMD5','CPI': 'C','Unemployment':'UN'}, inplace=True)

In [ ]:
lojas_preenchidas_p = lojas_p_features.merge(filling_p, on=['Store','IsHoliday_x','Type'], how='inner')

In [ ]:
lojas_preenchidas_p.MarkDown1.fillna(lojas_preenchidas_p['FMD1'],inplace=True)
lojas_preenchidas_p.MarkDown2.fillna(lojas_preenchidas_p['FMD2'],inplace=True)
lojas_preenchidas_p.MarkDown3.fillna(lojas_preenchidas_p['FMD3'],inplace=True)
lojas_preenchidas_p.MarkDown4.fillna(lojas_preenchidas_p['FMD4'],inplace=True)
lojas_preenchidas_p.MarkDown5.fillna(lojas_preenchidas_p['FMD5'],inplace=True)
lojas_preenchidas_p.CPI.fillna(lojas_preenchidas_p['C'],inplace=True)
lojas_preenchidas_p.Unemployment.fillna(lojas_preenchidas_p['UN'],inplace=True)
lojas_preenchidas_p.drop(['FMD1','FMD2','FMD3','FMD4','FMD5','C','UN'], axis=1, inplace=True)
lojas_preenchidas_p.info()

In [ ]:
lojas_feriados_p = lojas_preenchidas_p.merge(pd.DataFrame(feriados), on='Date', how='left')
lojas_feriados_p['TypeHoliday'].fillna(0,inplace=True)
lojas_feriados_p

In [ ]:
lojas_feriados_p['IsHoliday'] = pd.get_dummies(lojas_feriados_p.IsHoliday_x)[1]
lojas_feriados_p['SB'] = pd.get_dummies(lojas_feriados_p.TypeHoliday)['SB']
lojas_feriados_p['Labor'] = 0
lojas_feriados_p['Thanksgiving'] = pd.get_dummies(lojas_feriados_p.TypeHoliday)['Thanksgiving']
lojas_feriados_p['Christmas'] = pd.get_dummies(lojas_feriados_p.TypeHoliday)['Christmas']

lojas_feriados_p

In [ ]:
lojas_feriados_p['TypeA'] = pd.get_dummies(lojas_feriados_p.Type)['A']
lojas_feriados_p['TypeB'] = pd.get_dummies(lojas_feriados_p.Type)['B']
lojas_feriados_p['TypeC'] = pd.get_dummies(lojas_feriados_p.Type)['C']
lojas_feriados_p['Month'] = pd.to_datetime(lojas_feriados_p.Date).dt.month
lojas_feriados_p['Year'] =  pd.to_datetime(lojas_feriados_p.Date).dt.year

In [ ]:
colunas = X_train.columns

Aplicando as Predições

In [ ]:
lojas_predizer = lojas_feriados_p[colunas]
lojas_predizer.fillna(0, inplace=True)

In [ ]:
lojas_preenchidas_p['Weekly_Sales'] = rf.predict(lojas_predizer).round(2)

In [ ]:
predicoes_finais = lojas_preenchidas_p[['Store','Dept','Date','Weekly_Sales']]

In [ ]:
predicoes_finais['Id'] = (predicoes_finais[['Store','Dept','Date']].astype('str').apply('_'.join, axis=1))

In [ ]:
sample_submission = (f'{filepath}sampleSubmission.csv.zip')

In [ ]:
submission = predicoes_finais[['Id','Weekly_Sales']]

In [ ]:
submission.info()

In [ ]:
submission.to_csv('submission.csv', index=False)

# Conclusões

Analisamos as variáveis e as possíveis informações para o modelo e levantamos algumas hipóteses e conclusçoes:
<n>1) Para obter um baixo <b>WMAE</b> consideramos os departamentos como uma variável de alto poder de predição apra o modelo. Para isso assumimos que os números dos departamentos tem consistência entre as lojas, ou seja, o Dept 1 da loja 1 representa o mesmo departamento que o Dept 1 da loja 45. Uma outra forma de modelar seria utilizar a soma total das vendas por lojas e quebrar as vendas por departamento de acordo com a representatividade de cada departamento por loja. Fizemos a simulação e encontramos resultados interessantes, porém um pouco distantes das lojas do Tipo C.
<n>2) Caso fossesmo tratar com uma predição para lojas futuras, seria interessante não considerar a variável Loja e Departamento nas predições, afim de não viesar os resultados por lojas já conhecidas.
<n>3) Em caso de predições de vendas futuras, onde não conhecemos as features agregadas, poderiamos buscar a abordagem de séries temporais, e tentar o fit de um modelo <b>ARIMA</b> para os dados,para isso deveríamos usar a diferença de vendas semanais agregadas por loja, dado que essa variávei <b>Weekly_Sales.diff()</b> possui um comportamento estacionário no tmepo.